## Condy Kan

# Assignment 2: Pandas Data Management

### Concept
Data management contains multiple steps, including data cleaning and exploratory analysis. In this project, you will showcase skill in data management using Pandas.

### Data
You will use publicly available files. The first contains data on causes of death, while the second contains population data. Both files have state-level information for multiple years.

NCHS_-_Leading_Causes_of_Death__United_States
nst-est2018-01

### Requirements
To demonstrate pandas skills and ability, answer these questions:
Are Americans facing increasing, decreasing, or steady likelihood of death?
What are the four leading causes of death for Americans?
Do individual states show the same four leading causes of death?
Are there year-by-year changes in the four leading causes of death nationwide?
Use appropriately constructed and formatted tables to show results. There is no need to use visualization in this project.
Use population data appropriately to demonstrate your understanding of how variables are normalized/standardized.
Show skill in constructing a formal report using Jupyter.
Your formal report should contain components such as:

An introduction that discusses the scope of the analysis
A description of data used in the analysis along with data cleaning procedures
Code that clearly shows how an algorithm is implemented
Results
Discussion of results and generation of insight when appropriate
Summary when appropriate

In [1]:
import pandas as pd
import numpy as np 
dir ='C:/Users/Condy/Desktop/Spring2020/BANA680/'
myfile1 = 'NCHS_-_Leading_Causes_of_Death__United_States.csv'
deathDF= pd.read_csv(dir+myfile1)
deathDF.head(2)

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate
0,2012,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,21,2.6
1,2016,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,30,3.7


# From the code below, I cleaned the data such as dropping the regional areas, Puerto Rico from the State column. I removed United States from the State column because I already have the values for each state in America. I dropped Census and Estimates Base columns because we would not analyze them. Finally, I melted the dataframe. The years are transposed, along with Total Population

In [2]:
myfile2 = 'nst-est2018-01.xlsx'
dfB = pd.read_excel(dir + myfile2, skiprows=3).dropna()
dfB = dfB.rename(columns={"Unnamed: 0":"State"})
dfB = dfB[~dfB['State'].isin(['United States', 'Northeast', 'Midwest', 'South', 'West', 'Puerto Rico'])].drop(['Census', 'Estimates Base'], axis=1)
cols = dfB.select_dtypes(['object']) # identifies columns to strip
dfB[cols.columns] = cols.apply(lambda x : x.str.strip('.')) #strip away
dfB = dfB.melt(['State'],var_name='Year',value_name='Total Population')
dfB['Year'] = dfB['Year'].astype(int)
populationDF = dfB
populationDF.head(2)

,State,Year,Total Population
0,Alabama,2010,4785448.0
1,Alaska,2010,713906.0


# Merging two datasets (Death and Population) into one data. I used the inner join so it would show that what they have in common. It would share on the year between 2010 and 2016 and states.

In [3]:
mergedData = pd.merge(deathDF, populationDF, on=["Year","State"], how='inner')
mergedData
# Merging the death and population datasets so it would only show from 2010 to 2016. 

,Year,113 Cause Name,Cause Name,State,Deaths,Age-adjusted Death Rate,Total Population
0,2012,"Nephritis, nephrotic syndrome and nephrosis (N...",Kidney disease,Vermont,21,2.6,626063.0
1,2012,Influenza and pneumonia (J09-J18),Influenza and pneumonia,Vermont,81,9.9,626063.0
2,2012,"Intentional self-harm (suicide) (*U03,X60-X84,...",Suicide,Vermont,87,13.0,626063.0
3,2012,Diabetes mellitus (E10-E14),Diabetes,Vermont,168,21.4,626063.0
4,2012,Alzheimer's disease (G30),Alzheimer's disease,Vermont,284,35.3,626063.0
...,...,...,...,...,...,...,...
3922,2015,"Accidents (unintentional injuries) (V01-X59,Y8...",Unintentional injuries,Arkansas,1538,49.6,2978407.0
3923,2015,Chronic lower respiratory diseases (J40-J47),CLRD,Arkansas,2270,62.4,2978407.0
3924,2015,Malignant neoplasms (C00-C97),Cancer,Arkansas,6727,185.4,2978407.0
3925,2015,"Diseases of heart (I00-I09,I11,I13,I20-I51)",Heart disease,Arkansas,7938,223.2,2978407.0


## 1. Are Americans facing increasing, decreasing, or steady likelihood of death?

In [4]:
df1a = mergedData
keep = ['Year','Deaths', 'Cause Name', "Total Population"]
df1a = df1a[keep]
df1a = df1a.loc[(df1a['Cause Name'] == 'All causes')]
grouped = df1a.groupby(['Year','Cause Name'])
N1result = grouped[['Deaths','Total Population']].sum()
N1result['Rate of Likeihood of Death'] = (N1result['Deaths']/N1result['Total Population'])*100
N1result

,,Deaths,Total Population,Rate of Likeihood of Death
Year,Cause Name,,,
2010,All causes,2468435,309326085.0,0.798004
2011,All causes,2515458,311580009.0,0.807323
2012,All causes,2543279,313874218.0,0.810286
2013,All causes,2596993,316057727.0,0.821683
2014,All causes,2626418,318386421.0,0.824915
2015,All causes,2712630,320742673.0,0.845734
2016,All causes,2744248,323071342.0,0.849425


For Number 1, based on the output above, the Rate of Likeihood of Death is slightly increased from 2010 to 2016 due to the merging data of the death and population datasets. Overall, I believe that Americans are facing a slightly increasing likeihood of death.

# 2. What are the four leading causes of death for Americans?

In [6]:
df2a = mergedData
keep = ['Cause Name', 'Deaths']
df2a = df2a[keep]
df2a = df2a.loc[(df2a['Cause Name'] != 'All causes')]
grouped = df2a.groupby(['Cause Name'])
N2result = grouped[['Deaths']].sum()
N2result = N2result.sort_values(by=('Deaths'), ascending=False).iloc[0:4,] # top four leadiing cancers
N2result

,Deaths
Cause Name,
Heart disease,4288532
Cancer,4104606
CLRD,1030455
Unintentional injuries,949519


For Number 2, I used the merged data to find the four leading causes of death for Americans. Based on the output above, the four leading causes of deaths for Americans are Heart disease, Cancer, Stroke, and Unintentional injuries respectively from 2010 to 2016. 

# 3 Do individual states show the same four leading causes of death?

In [8]:
pd.set_option('display.max_rows', None)
df3a = mergedData
keep = ['State','Cause Name', 'Deaths']
df3a = df3a[keep]
df3a = df3a.loc[(df3a['Cause Name'] != 'All causes')]
df3a['Result'] = df3a['Cause Name'].isin(['Heart disease', 'Cancer', 'CLRD', 'Unintentional injuries'])
grouped = df3a.groupby(['State','Cause Name'])
df3a = grouped[['Deaths']].sum()
df3a = df3a.sort_values(by=['State',('Deaths')], ascending=[True, False])
df3a = df3a.groupby('State').head(4)
df3a

Deaths
State                Cause Name                    
Alabama              Heart disease            86807
                     Cancer                   72092
                     CLRD                     21491
                     Stroke                   18986
Alaska               Cancer                    6705
                     Heart disease             5319
                     Unintentional injuries    2678
                     CLRD                      1390
Arizona              Cancer                   78907
                     Heart disease            76280
                     CLRD                     23535
                     Unintentional injuries   23363
Arkansas             Heart disease            52835
                     Cancer                   46085
                     CLRD                     14289
                     Stroke                   11613
California           Heart disease           419134
                     Cancer                  405848
                     Stroke                   98881
                     CLRD                     93016
Colorado             Cancer                   51686
                     Heart disease            46168
                     Unintentional injuries   17383
                     CLRD                     16489
Connecticut          Heart disease            49979
                     Cancer                   47074
                     Unintentional injuries   11068
                     CLRD                      9601
Delaware             Cancer                   13761
                     Heart disease            13071
                     CLRD                      3315
                     Stroke                    3049
District of Columbia Heart disease             9031
                     Cancer                    7519
                     Unintentional injuries    1690
                     Stroke                    1507
Florida              Heart disease           303048
                     Cancer                  299576
                     CLRD                     77433
                     Unintentional injuries   67853
Georgia              Heart disease           117154
                     Cancer                  114288
                     CLRD                     29546
                     Unintentional injuries   27996
Hawaii               Heart disease            17123
                     Cancer                   16516
                     Stroke                    4559
                     Unintentional injuries    3463
Idaho                Cancer                   18910
                     Heart disease            18497
                     CLRD                      5640
                     Unintentional injuries    5199
Illinois             Heart disease           175141
                     Cancer                  170733
                     CLRD                     38273
                     Stroke                   38208
Indiana              Heart disease            95872
                     Cancer                   93424
                     CLRD                     28541
                     Stroke                   21384
Iowa                 Heart disease            47981
                     Cancer                   45235
                     CLRD                     13179
                     Unintentional injuries   10095
Kansas               Cancer                   38300
                     Heart disease            38265
                     CLRD                     11618
                     Unintentional injuries    9624
Kentucky             Cancer                   70698
                     Heart disease            70171
                     CLRD                     22117
                     Unintentional injuries   19271
Louisiana            Heart disease            72738
                     Cancer                   65164
                     Unintentional injuries   16417
                     Stroke              

In [8]:
N3result = df3a.reset_index()
N3result = N3result.groupby('Cause Name')['State'].agg(['count'])
N3result

,count
Cause Name,
Alzheimer's disease,2
CLRD,45
Cancer,51
Heart disease,51
Stroke,20
Unintentional injuries,35


For Number 3, I used the merged data so it would show the individual states based from 2010 to 2016. Based on the output above, not all the individual states show the same four leading causes of death. For example, Alabama and Alaska do not display the same four leading causes in death. Alabama has Stroke, not Unintentional injuries between 2010 to 2016 from the dataframe above.


Alabama, Arkansas, California, Delware, District of Columbia, Hawaii, Illinois, Indiana, Louisiana, Maryland, Michigan, Nebraska, New Jersey, New York, North Carolina, North Dakota, Oregon, Pennsylvania, Texas, Utah, Virginia, and Washington do not display the same four leading causes of death from 2010 to 2016.  

To double check, I used the groupby function to get count values for Cause Name in States. From the second table above with the count column, it shows that two states have Alzheimzer's disease, 45 states have CLRD, 51 states have cancer and heart disease, 20 states has stroke, and 35 states has unintentional injuries from 2010 to 2016. 

# 4.Are there year-by-year changes in the four leading causes of death nationwide? 

In [9]:
df4a = mergedData
keep = ['Year','Cause Name', 'Deaths']
df4a = df4a[keep]
df4a = df4a.loc[(df4a['Cause Name'] != 'All causes')]
grouped = df4a.groupby(["Year",'Cause Name'])
df4a = grouped[['Deaths']].sum()
df4a = df4a.sort_values(by=['Year', ('Deaths')], ascending=[True, False])
df4a = df4a.groupby('Year').head(4)
df4a

Deaths
Year Cause Name                    
2010 Heart disease           597689
     Cancer                  574743
     CLRD                    138080
     Stroke                  129476
2011 Heart disease           596577
     Cancer                  576691
     CLRD                    142943
     Stroke                  128932
2012 Heart disease           599711
     Cancer                  582623
     CLRD                    143489
     Stroke                  128546
2013 Heart disease           611105
     Cancer                  584881
     CLRD                    149205
     Unintentional injuries  130557
2014 Heart disease           614348
     Cancer                  591700
     CLRD                    147101
     Unintentional injuries  135928
2015 Heart disease           633842
     Cancer                  595930
     CLRD                    155041
     Unintentional injuries  146571
2016 Heart disease           635260
     Cancer                  598038
     Unintentional injuries  161374
     CLRD                    154596

For Number 4, based on the output above, there are year-by-year changes in the four leading causes of death nationwide from 2010 to 2016 because of the merged data. From 2010 to 2012, the four leading causes of death : heart disease, cancer, CLRD, and Stroke, in the order were the same. From 2013 to 2015, stroke was not one of the four leading causes of death anymore so unintentional injuries has become fourth leading causes of death. Lastly, the unintentional injuries moved up to third leading cause of death in 2016.

Overall, heart disease and cancer have been top two leading causes nationwide. CLRD was still one of those four leading causes from 2010 to 2012. Stroke was one of the four leading causes from 2010 to 2012 then unintentional injuries became one of the four leading causes from 2013 to 2016. 

## Summary: 

From this assignment, I learned how to clean and merge data within the Pandas libary. I understand the importance of using inner for the merged data that helps with cleaning data. PLus, using the melt function and merged data allows me construct and format tables to display results appropriately. I developed and improved my skills in python programming that would manage data much better. Also, I gained the understanding of function uses such as groupby, isin, drop, sort_value and loc so I could analyze and show the results as I wanted. This assignment is different from the last assignment because we were limited to use python and numpy libaries in assignment 1. Now, we are allowed to use pandas that allow me create dataframes and handle data more. Overall, I became more familar with the pandas' libary. I am looking forward to working with Pandas in future assginemts.